In [3]:
import os
import numpy as np
import torch
from torchvision import datasets, models, transforms
from tqdm import tqdm
from PIL import Image
from torch.autograd import Variable

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report, roc_curve, auc
from efficientnet_pytorch import EfficientNet

In [11]:
data_dir = 'Input_dataset'
class_names = ['Diptera', 'Tephritidae']
# train_path = data_dir +'/train/'
# val_path = data_dir +'/val/'
# test_path = data_dir +'/test/'
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [12]:
# model = torch.hub.load('pytorch/vision:v0.9.0', 'resnet50', pretrained=True)
# layer = model._modules.get('avgpool')

Using cache found in C:\Users\dukes/.cache\torch\hub\pytorch_vision_v0.9.0


In [ ]:
model = torch.hub.load('pytorch/vision:v0.9.0', 'resnet152', pretrained=True)
layer = model._modules.get('avgpool')

In [13]:
model.eval

<bound method Module.eval of ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, ke

In [14]:
scaler = transforms.Resize((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()

In [16]:
# contains the hog and color features from images
features_list = []
# labels
Y = []
# number of classes
l = len(class_names)

# Function to Extract features from the images
def create_feature_matrix(folderName):
    sourceFiles=os.listdir(data_dir +'/'+ folderName)
    for i in tqdm(sourceFiles):
        # load image
        file_path = os.path.join(data_dir +'/'+ folderName +'/'+i)
        img = Image.open(file_path).convert("RGB")
        t_img = Variable(normalize(to_tensor(scaler(img))).unsqueeze(0))
#         print(np.array(img).shape)
#         print(t_img.shape)
#         print(t_img)

        # layer_output_size = 2048 for ResNet50/ ResNeSt50
        # layer_output_size = 1280 for EfficientNet-B0
        features = torch.zeros(1, 2048, 1, 1)
        def copy_data(m,i,o):
            features.copy_(o.data)
        h = layer.register_forward_hook(copy_data)
        model(t_img)
        h.remove()
        features = features.numpy()[:, :, 0, 0]
        features_list.append(features)
        
        for j in range(l):
            if (folderName==class_names[j]):
                Y.append(j)
              
                
# Organize file names and class labels in X and Y variables
for j in range(l):
    create_feature_matrix(class_names[j])  
    
# convert list of arrays into a matrix     
X_features = np.array(features_list)
Y = np.array(Y) 

100%|██████████████████████████████████████████████████████████████████████████████| 5287/5287 [19:14<00:00,  4.58it/s]


In [17]:
X = X_features.reshape((X_features.shape[0],X_features.shape[2]))
print(X.shape)
print(Y.shape)

(50390, 2048)
(50390,)


In [28]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,
                                            stratify=Y, 
                                            test_size= 1/10)
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(45351, 2048)
(45351,)
(5039, 2048)
(5039,)


In [29]:
# svm_model = SVC(kernel='rbf')
# svm_model.fit(X_train, Y_train)

SVC()

In [30]:
wclf = SVC(kernel='rbf', class_weight={1: 8.5})
wclf.fit(X_train, Y_train)

SVC(class_weight={1: 8.5})

In [31]:
# Y_pred = svm_model.predict(X_test)
Y_predw = wclf.predict(X_test)

In [32]:
# print(confusion_matrix(Y_test, Y_pred))
# print("\n")
# print(classification_report(Y_test, Y_pred, target_names= class_names))

# print("Training set score for SVM: %f" % svm_model.score(X_train , Y_train))
# print("Testing  set score for SVM: %f" % svm_model.score(X_test  , Y_test))

# svm_model.score

[[4510    0]
 [ 529    0]]


              precision    recall  f1-score   support

     Diptera       0.90      1.00      0.94      4510
 Tephritidae       0.00      0.00      0.00       529

    accuracy                           0.90      5039
   macro avg       0.45      0.50      0.47      5039
weighted avg       0.80      0.90      0.85      5039



C:\Users\dukes\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\dukes\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\dukes\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Training set score for SVM: 0.895085
Testing  set score for SVM: 0.895019


<bound method ClassifierMixin.score of SVC()>

In [33]:
print(confusion_matrix(Y_test, Y_predw))
print("\n")
print(classification_report(Y_test, Y_predw, target_names= class_names))

print("Training set score for SVM: %f" % wclf.score(X_train , Y_train))
print("Testing  set score for SVM: %f" % wclf.score(X_test  , Y_test))

wclf.score

[[3462 1048]
 [ 143  386]]


              precision    recall  f1-score   support

     Diptera       0.96      0.77      0.85      4510
 Tephritidae       0.27      0.73      0.39       529

    accuracy                           0.76      5039
   macro avg       0.61      0.75      0.62      5039
weighted avg       0.89      0.76      0.80      5039

Training set score for SVM: 0.800820
Testing  set score for SVM: 0.763644


<bound method ClassifierMixin.score of SVC(class_weight={1: 8.5})>

In [34]:
print('Best C:',wclf.C,"\n") 
print('Best Kernel:',wclf.kernel,"\n")
print('Best Gamma:',wclf.gamma,"\n")

Best C: 1.0 

Best Kernel: rbf 

Best Gamma: scale 



In [19]:
params_grid = [{'kernel': ['rbf'], 'gamma': [0.5, 0.01, 1e-3, 1e-4],
                     'C': [1, 10, 100]}]
# A low C makes the decision surface smooth, while a high C aims at classifying 
# all training examples correctly.

# Performing CV to tune parameters for best SVM fit 
svm_model = GridSearchCV(SVC(), params_grid, cv=5)
svm_model.fit(X_train, Y_train)

GridSearchCV(cv=5, estimator=SVC(),
             param_grid=[{'C': [1, 10, 100],
                          'gamma': [0.5, 0.01, 0.001, 0.0001],
                          'kernel': ['rbf']}])

In [20]:
# View the accuracy score
print('Best score for training data:', svm_model.best_score_,"\n") 

# View the best parameters for the model found using grid search
print('Best C:',svm_model.best_estimator_.C,"\n") 
print('Best Kernel:',svm_model.best_estimator_.kernel,"\n")
print('Best Gamma:',svm_model.best_estimator_.gamma,"\n")

final_model = svm_model.best_estimator_
Y_pred = final_model.predict(X_test)
# Y_pred_label = list(encoder.inverse_transform(Y_pred))

Best score for training data: 0.9057472799056174 

Best C: 10 

Best Kernel: rbf 

Best Gamma: 0.5 



In [21]:
# Making the Confusion Matrix
#print(pd.crosstab(Y_test_label, Y_pred_label, rownames=['Actual Activity'], colnames=['Predicted Activity']))
print(confusion_matrix(Y_test, Y_pred))
print("\n")
print(classification_report(Y_test, Y_pred, target_names= class_names))

print("Training set score for SVM: %f" % final_model.score(X_train , Y_train))
print("Testing  set score for SVM: %f" % final_model.score(X_test  , Y_test))

svm_model.score

[[11170   106]
 [ 1089   233]]


              precision    recall  f1-score   support

     Diptera       0.91      0.99      0.95     11276
 Tephritidae       0.69      0.18      0.28      1322

    accuracy                           0.91     12598
   macro avg       0.80      0.58      0.61     12598
weighted avg       0.89      0.91      0.88     12598

Training set score for SVM: 0.999974
Testing  set score for SVM: 0.905144


<bound method BaseSearchCV.score of GridSearchCV(cv=5, estimator=SVC(),
             param_grid=[{'C': [1, 10, 100],
                          'gamma': [0.5, 0.01, 0.001, 0.0001],
                          'kernel': ['rbf']}])>

In [35]:
# training a KNN classifier
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 4).fit(X_train, Y_train)
  
# accuracy on X_test
accuracy = knn.score(X_test, Y_test)
print(accuracy)
  
# creating a confusion matrix
knn_predictions = knn.predict(X_test) 
cm = confusion_matrix(Y_test, knn_predictions)
print(cm)
print("\n")
print(classification_report(Y_test, knn_predictions, target_names= class_names))


0.8954157570946616
[[4502    8]
 [ 519   10]]


              precision    recall  f1-score   support

     Diptera       0.90      1.00      0.94      4510
 Tephritidae       0.56      0.02      0.04       529

    accuracy                           0.90      5039
   macro avg       0.73      0.51      0.49      5039
weighted avg       0.86      0.90      0.85      5039



In [36]:
# training a Naive Bayes classifier
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB().fit(X_train, Y_train)
gnb_predictions = gnb.predict(X_test)
  
# accuracy on X_test
accuracy = gnb.score(X_test, Y_test)
print(accuracy)
  
# creating a confusion matrix
cm = confusion_matrix(Y_test, gnb_predictions)
print(cm)
print("\n")
print(classification_report(Y_test, gnb_predictions, target_names= class_names))

0.7328835086326652
[[3328 1182]
 [ 164  365]]


              precision    recall  f1-score   support

     Diptera       0.95      0.74      0.83      4510
 Tephritidae       0.24      0.69      0.35       529

    accuracy                           0.73      5039
   macro avg       0.59      0.71      0.59      5039
weighted avg       0.88      0.73      0.78      5039



In [37]:
# training a Naive Bayes classifier
from sklearn.naive_bayes import ComplementNB
cmb = ComplementNB().fit(X_train, Y_train)
cmb_predictions = cmb.predict(X_test)
  
# accuracy on X_test
accuracy = cmb.score(X_test, Y_test)
print(accuracy)
  
# creating a confusion matrix
cm = confusion_matrix(Y_test, cmb_predictions)
print(cm)
print("\n")
print(classification_report(Y_test, cmb_predictions, target_names= class_names))

0.6793014487001389
[[3041 1469]
 [ 147  382]]


              precision    recall  f1-score   support

     Diptera       0.95      0.67      0.79      4510
 Tephritidae       0.21      0.72      0.32       529

    accuracy                           0.68      5039
   macro avg       0.58      0.70      0.56      5039
weighted avg       0.88      0.68      0.74      5039



In [38]:
# training a Naive Bayes classifier
from sklearn.naive_bayes import MultinomialNB
cmb = MultinomialNB().fit(X_train, Y_train)
cmb_predictions = cmb.predict(X_test)
  
# accuracy on X_test
accuracy = cmb.score(X_test, Y_test)
print(accuracy)
  
# creating a confusion matrix
cm = confusion_matrix(Y_test, cmb_predictions)
print(cm)
print("\n")
print(classification_report(Y_test, cmb_predictions, target_names= class_names))

0.8950188529470133
[[4510    0]
 [ 529    0]]


              precision    recall  f1-score   support

     Diptera       0.90      1.00      0.94      4510
 Tephritidae       0.00      0.00      0.00       529

    accuracy                           0.90      5039
   macro avg       0.45      0.50      0.47      5039
weighted avg       0.80      0.90      0.85      5039



C:\Users\dukes\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\dukes\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\dukes\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
